<a href="https://colab.research.google.com/github/MichiganDataScienceTeam/F25-Shazam/blob/main/notebooks/week1_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install numpy matplotlib librosa scipy yt-dlp

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import scipy

## Terminology

Explain Frequency, discretization of audio:

1 Hertz = 1 vibration / second 

vibration of air particles

bit depth: number of bits used to store each sample

sample rate: number of samples per second (Hz)

time is digitized using the sample rate, amplitude is digitized using bit depth

$\mathrm{bit rate} = \mathrm{sr}\times\mathrm{bit depth}\times\textrm{number of channels}$

mp3 files are measured in bit rate since they do not store the associated bit depths

The sampling interval is defined as the reciprocal of the sampling rate, and it represents the time duration between two consecutive samples in a discrete time signal. If the sampling rate is 44100 times per second, then the sampling interval is 1/44100.


try different values of `sample_rate` and `freq_Hz`

In [ ]:
# frequency of the audio signal
freq_Hz = 5

# rate at which audio is being sampled
sample_rate_Hz = 10

x_continuous = np.linspace(0, 1, 1000, endpoint=False)
y_continuous = np.cos(freq_Hz * 2*np.pi*x_continuous)

x_discrete = np.linspace(0, 1, sample_rate_Hz, endpoint=False)
y_discrete = np.cos(freq_Hz * 2*np.pi*x_discrete)

plt.plot(x_continuous, y_continuous)
plt.scatter(x_discrete, y_discrete, color="red", s=50)
plt.xlabel("Time (s)")
plt.grid()


## Good to Know Sampling Theorem:

If a signal contains no frequencies higher than $f_\mathrm{max}$, then the signal can be perfectly reconstructed when sampled at a rate $sr > 2f_\mathrm{max}$. In other words, the maximum reconstructable frequency is strictly less than $sr/2$. This is called the Nyquist-Shannon sampling theorem.

Explain aliasing briefly

In [ ]:
sample_rate_Hz = 2

freq_Hz = 5

freq_reconstructable_up_to = sample_rate_Hz / 2

x_continuous = np.linspace(0, 1, 1000, endpoint=False)
y_continuous = np.cos(freq_Hz * 2*np.pi*x_continuous)

x_undersampled = np.linspace(0, 1, 1000, endpoint=False)
y_undersampled = np.cos(freq_reconstructable_up_to * 2*np.pi*x_undersampled)

x_discrete = np.linspace(0, 1, sample_rate_Hz, endpoint=False)
y_discrete = np.cos(freq_Hz * 2*np.pi*x_discrete)

plt.plot(x_continuous, y_continuous)
plt.plot(x_undersampled, y_undersampled, color="red", linestyle="--")
plt.scatter(x_discrete, y_discrete, color="red", s=50)
plt.xlabel("Time (s)")
plt.grid()

## Working with Audio Data

In [ ]:
import yt_dlp
import os

# add any youtube video url here, 
# copied from browser address bar
youtube_url = ""

yt_audio_path = "yt_sample.wav"

ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': "yt_sample.%(ext)s",  # output file
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',     # save as wav file
        }],
        #'cookiefile': 'cookies.txt',
    }
if youtube_url != "":
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])



In [ ]:
#audio_path = yt_audio_path
audio_path = "sample.wav"

# default is to convert to mono (1 channel)
audio, sr = librosa.load(audio_path, sr=None)  # sr=None uses the file's sampling rate
print(f"sample rate = {sr} Hz")

plt.figure(figsize=(12, 4))
plt.plot(audio)
plt.title("Waveform of Audio")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

print("audio is digitally represented as a numpy.ndarray:")
audio

In [ ]:
# Resampling:
# internally, downsampling (reducing the sample rate) uses a low pass filter to remove higher frequency components to avoid aliasing (Sampling Theorem)
# then keeps every nth sample using a sample step size of (old_sr/new_sr), first upsampling via interpolation if old_sr/new_sr is not an integer
sr = 11_025
audio, sr = librosa.load(audio_path, sr=sr)

# Question: how would you plot just the first 10 seconds of an audio file?
#            Modify the existing code to do this
# Hint: use sample rate

plt.figure(figsize=(12, 4))
plt.plot(audio, lw=0.3)
plt.scatter(np.arange(len(audio)), audio, s=2, color="red")
plt.title("Resampled Audio Waveform")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()
audio

## Next week: visualizing frequency using a spectrogram

## Method 1: Librosa

In [ ]:
audio, sr = librosa.load(audio_path, sr=None) 

# parameters of the short-time Fourier transform:
# (algorithm that creates the spectrogram)
win_length = 2**11  # number of samples in each window
n_fft = win_length
hop_length = win_length // 4
window = scipy.signal.get_window("triang", Nx=win_length)

S = librosa.stft(audio, 
                       n_fft=n_fft, hop_length=hop_length, 
                       win_length=win_length, window=window)
S_magnitude = np.abs(S)  # |a+bi| = sqrt(a^2 + b^2)
S_db = librosa.amplitude_to_db(S_magnitude, ref=np.max)

im = plt.imshow(S_db, cmap="inferno", aspect="auto", 
                origin="lower")
plt.colorbar(im, format="%+2.0f dB")
plt.xlabel("Time (sec)")
plt.ylabel("Frequency (Hz)")
plt.show()

## Method 2: Scipy (what we'll use)

In [ ]:
audio, sr = librosa.load(audio_path, sr=None) 

# parameters of the short-time Fourier transform:
# (algorithm that creates the spectrogram)
nperseg = win_length = 2**11  # number of samples in each window
nfft = n_fft = win_length
hop_length = win_length // 4
window = scipy.signal.get_window("triang", Nx=win_length)

# scipy.signal.stft also uses the sample rate to output 
# frequency (in Hz) and time (in seconds) vectors,
# corresponding to the rows and columns of the stft matrix 
# in "s_scipy"
fs=sr 
noverlap = nperseg - hop_length

freq_scipy, time_scipy, s_scipy = scipy.signal.stft(
    audio, 
    fs=fs, window="hann", nfft=nfft, 
    nperseg=nperseg, noverlap=noverlap
)

print(f"freq vector shape: {freq_scipy.shape}")
print(f"time vector shape: {time_scipy.shape}")
print(f"stft matrix shape: {s_scipy.shape}")

s_scipy_db = librosa.amplitude_to_db(np.abs(s_scipy), ref=np.max)

im = plt.imshow(s_scipy_db, cmap="inferno", aspect="auto", 
                origin="lower")
plt.colorbar(im, format="%+2.0f dB")
plt.xlabel("Time (sec)")
plt.ylabel("Frequency (Hz)")
plt.show()